In [1]:
!pip install pennylane

In [2]:
import pennylane as qml
import numpy as np
import torch
import torch.nn as nn


n_qubits = 5

# define o simulador de estados
dev = qml.device("default.qubit", wires=n_qubits)

device = 'cpu' # executaremos tudo em cpu

def H_layer(nqubits):
    """Camada de portas Hadamard de um único qubit."""
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)


def RY_layer(w):
    """Camada de rotações parametrizadas de qubit ao redor do eixo y."""
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)


def entangling_layer(nqubits):
    """Camada de CNOTs seguida por outra camada deslocada de CNOT (gera emaranhamento)."""
    # Em outras palavras, teremos algo como :
    # CNOT  CNOT  CNOT  CNOT...  CNOT
    #   CNOT  CNOT  CNOT...  CNOT
    for i in range(0, nqubits - 1, 2):  # Iteramos sobre os indices pares: i=0,2,...N-2
        qml.CNOT(wires=[i, i + 1])
    for i in range(1, nqubits - 1, 2):  # Iteramos sobre os indices impares:  i=1,3,...N-3
        qml.CNOT(wires=[i, i + 1])

@qml.qnode(dev, interface="torch")
def quantum_net(q_input_features, q_weights_flat, q_depth, n_qubits):
    """
    Nossa Rede Neural Quântica (RNQ).
    """

    # Apenas ajeitamos a dimensão dos pesos para profundidade x n_qubits.
    q_weights = q_weights_flat.reshape(q_depth, n_qubits)

    # Sequência de camadas parametrizadas.
    for k in range(q_depth):
        # U_0(x) : camada de codificação nas amplitudes do estado.
        qml.AmplitudeEmbedding(features=q_input_features, normalize=True, pad_with=0, wires=range(n_qubits))

        # camada de emaranhamento seguida de rotações parametrizadas em Y.
        entangling_layer(n_qubits)
        RY_layer(q_weights[k])

    # Retorna o valor esperado em Z
    exp_vals = [qml.expval(qml.PauliZ(position)) for position in range(n_qubits)]
    return tuple(exp_vals)



class RNQ(nn.Module):
    """
    Módulo Torch que implementa nossa rede quântica híbrida.
    """

    def __init__(self, q_depth, n_qubits, q_delta):
        """
        Inicialização. Perceba que post_net é uma camada clássica ao final do circuito,
        projetando a medida em 2 neurônios clássicos, que representarão a probabilidade
        do paciente ser patológico ou não.
        """

        super().__init__()
        self.n_qubits = n_qubits
        self.q_depth = q_depth
        self.q_params = nn.Parameter(q_delta * torch.randn(q_depth * n_qubits))
        self.post_net = nn.Linear(n_qubits, 2)

    def forward(self, input_features):
        """
        No forward, definimos como os tensores se movem no modelo.
        """

        # Obtém as features de entrada para o circuito quântico
        # (observe a normalização necessária)
        q_in = torch.tanh(input_features) * np.pi / 2.0

        # Aplica o circuito quântico em cada elemento do batch e anexa em q_out
        q_out = torch.Tensor(0, n_qubits)
        q_out = q_out.to(device)
        for elem in q_in:
            q_out_elem = torch.tensor(quantum_net(elem, self.q_params, self.q_depth, self.n_qubits))
            q_out = torch.cat((q_out, q_out_elem.unsqueeze(0)))

        # Retorna a predição da camada clássica de pós-processamento
        return self.post_net(q_out.to(torch.float32))

In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

df = pd.read_csv('parkinsons.txt')

features=df.loc[:,df.columns!='status'].values[:,1:]
labels=df.loc[:,'status'].values

scaler = MinMaxScaler()

In [4]:
X_train,X_test,y_train,y_test=train_test_split(features, labels, stratify=labels, test_size=0.2, random_state=42)

In [5]:
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [6]:
import torch
import torch.utils.data as data_utils

batch_size = 32

train_ds = data_utils.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
train_loader = data_utils.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_ds = data_utils.TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
test_loader = data_utils.DataLoader(test_ds, batch_size=batch_size, shuffle=True)

dataloaders = {"train": train_loader, "validation": test_loader}
dataset_sizes = {"train": len(X_train), "validation": len(X_test)}

In [13]:
import torch
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

device = 'cpu'

def train_model(model, dataloaders, dataset_sizes, batch_size, criterion, optimizer, num_epochs):
    best_acc = 0.0
    best_f1 = 0.0
    best_loss = 10000.0  # número grande arbitrário
    best_acc_train = 0.0
    best_loss_train = 10000.0  # número grande arbitrário

    for epoch in range(num_epochs):

        # Cada época tem uma fase de treino
        # e outra de validação.
        for phase in ["train", "validation"]:
            if phase == "train":
                # Colocamos o modelo em modo de treinamento.
                model.train()
            else:
                # De modo alternativo, podemos colocá-lo
                # em modo de 'evaluation'
                model.eval()
            running_loss = 0.0
            running_corrects = 0

            # Iteramos sobre os dados
            n_batches = dataset_sizes[phase] // batch_size
            it = 0
            for inputs, labels in dataloaders[phase]:
                batch_size_ = len(inputs)
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                # Utilizaremos os gradientes apenas durante o treino.
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * batch_size_
                batch_corrects = torch.sum(preds == labels.data).item()
                running_corrects += batch_corrects
                it += 1

            # Mostramos os resultados
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]
            epoch_f1 = f1_score(labels.cpu(), preds.cpu(), average='weighted')
            print(
                "Estágio: {} Época: {}/{} Loss: {:.4f} Acc: {:.4f} F1: {:.4f}       ".format(
                    "train" if phase == "train" else "validation  ",
                    epoch + 1,
                    num_epochs,
                    epoch_loss,
                    epoch_acc,
                    epoch_f1
                )
            )

            # Salva as métricas da melhor época
            if phase == "validation" and epoch_acc > best_acc:
                best_acc = epoch_acc
            if phase == "validation" and epoch_f1 > best_f1:
                best_f1 = epoch_f1


            if phase == "validation" and epoch_loss < best_loss:
                best_loss = epoch_loss
            if phase == "train" and epoch_acc > best_acc_train:
                best_acc_train = epoch_acc
            if phase == "train" and epoch_loss < best_loss_train:
                best_loss_train = epoch_loss

    return best_acc, best_f1

In [16]:
import torch.optim as optim

device = 'cpu'
n_qubits = 5
q_depth = 3
step = 0.001
q_delta = 0.01

torch.manual_seed(42)

F1 = []
Accuracies = []
Recall = []
Spec = []

for q_depth in [4]:

  print("[*] Treinamento com {} camadas... \n".format(q_depth))
  rnq = RNQ(q_depth, n_qubits, q_delta)

  # podemos utilizar 'cuda', se houver GPU disponível, ou 'cpu'
  rnq = rnq.to(device)

  criterion = nn.CrossEntropyLoss()

  optimizer_hybrid = optim.Adam(rnq.parameters(), lr=step)

  acc, f1 = train_model(
      rnq, dataloaders, dataset_sizes, batch_size, criterion, optimizer_hybrid, num_epochs=28
  )
  F1.append(f1)
  Accuracies.append(acc)

[*] Treinamento com 4 camadas... 

Estágio: train Época: 1/28 Loss: 0.7143 Acc: 0.4359 F1: 0.5444       
Estágio: validation   Época: 1/28 Loss: 0.7197 Acc: 0.3077 F1: 0.1429       
Estágio: train Época: 2/28 Loss: 0.7093 Acc: 0.4359 F1: 0.4029       
Estágio: validation   Época: 2/28 Loss: 0.7145 Acc: 0.3590 F1: 0.1429       
Estágio: train Época: 3/28 Loss: 0.7042 Acc: 0.4744 F1: 0.5742       
Estágio: validation   Época: 3/28 Loss: 0.7095 Acc: 0.4872 F1: 0.5195       
Estágio: train Época: 4/28 Loss: 0.6993 Acc: 0.4872 F1: 0.5714       
Estágio: validation   Época: 4/28 Loss: 0.7047 Acc: 0.5385 F1: 0.4156       
Estágio: train Época: 5/28 Loss: 0.6949 Acc: 0.5256 F1: 0.5397       
Estágio: validation   Época: 5/28 Loss: 0.6998 Acc: 0.5641 F1: 0.7143       
Estágio: train Época: 6/28 Loss: 0.6901 Acc: 0.5769 F1: 0.5000       
Estágio: validation   Época: 6/28 Loss: 0.6952 Acc: 0.5641 F1: 0.3429       
Estágio: train Época: 7/28 Loss: 0.6855 Acc: 0.5705 F1: 0.5755       
Estágio: vali